In [2]:
import pandas as pd
import numpy as np
from nba_py import player, draftcombine
import datetime


In [3]:
now = datetime.datetime.now()
yearsSince2000 = now.year -2000

In [4]:
def get_measurements(year):
    resp = draftcombine.Summary(season=year)
    headers = resp.json['resultSets'][0]['headers']
    vals = resp.json['resultSets'][0]['rowSet']
    return pd.DataFrame(vals, columns=headers)
     

In [5]:
df =  get_measurements("2000-01" )
prospects = get_measurements("2018-19")

years = ["20{}-{}".format("{:02}".format(i), "{:02}".format(i+1)) for i in range(1,yearsSince2000)]
for y in years:
    df = df.append(get_measurements(y))

In [6]:
df =  get_measurements("2000-01" )
df.head()

,SEASON,PLAYER_ID,FIRST_NAME,LAST_NAME,PLAYER_NAME,POSITION,HEIGHT_WO_SHOES,HEIGHT_WO_SHOES_FT_IN,HEIGHT_W_SHOES,HEIGHT_W_SHOES_FT_IN,...,STANDING_REACH_FT_IN,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,MODIFIED_LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,BENCH_PRESS
0,2000,2124,Malik,Allen,Malik Allen,PF-C,80.25,6' 8.25'',None,None,...,9' 1'',None,None,None,25.5,29.0,11.83,None,3.38,13.0
1,2000,12019,Harold,Arceneaux,Harold Arceneaux,SG-SF,76.50,6' 4.5'',None,None,...,8' 7'',None,None,None,NaN,29.0,13.80,None,NaN,0.0
2,2000,12020,Lamont,Barnes,Lamont Barnes,PF-C,80.50,6' 8.5'',None,None,...,9' 0'',None,None,None,28.0,29.5,12.30,None,3.40,10.0
3,2000,12131,Mario,Bland,Mario Bland,PF,77.50,6' 5.5'',None,None,...,8' 7'',None,None,None,27.0,31.0,13.04,None,3.47,15.0
4,2000,2056,Primoz,Brezec,Primoz Brezec,C,84.75,7' 0.75'',None,None,...,9' 2'',None,None,None,26.0,29.5,11.53,None,3.55,NaN


In [87]:
from sklearn import neighbors
from sklearn.preprocessing import Imputer

numTypeCols = df.select_dtypes(include=[np.number]).columns.values
X = df[numTypeCols]
y = df["PLAYER_ID"]
print(X)
# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
imp = imp.fit(X)
XImp = imp.transform(X)
print(XImp)
clf = neighbors.KNeighborsClassifier(3, weights='distance')
clf.fit(XImp, y)

# numTypeCols = prospects.select_dtypes(include=[np.number]).columns.values
X2 = prospects[numTypeCols]

imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
imp = imp.fit(X2)
X2Imp = imp.transform(X2)
print(X2Imp)
clf.predict(X2Imp)

    PLAYER_ID  HEIGHT_WO_SHOES  WINGSPAN  STANDING_REACH  \
0        2124            80.25     86.50           109.0   
1       12019            76.50     80.50           103.0   
2       12020            80.50     87.50           108.0   
3       12131            77.50     84.00           103.0   
4        2056            84.75     86.00           110.0   
5        2081            82.00     85.25           109.5   
6        2049            70.50     72.00            94.5   
7       12132            76.00     83.00           102.0   
8       12133            72.25     76.00            96.5   
9       12134            76.25     79.00           100.0   
10       2037            76.50     82.00           102.5   
11      12017            79.75     86.00           107.5   
12       2064            69.00     73.50            93.0   
13      12018            81.75     82.50           106.5   
14       2238            81.25     80.50           106.0   
15       2109            71.75     76.00

array([12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135, 12135,
       12135, 12135, 12135, 12135, 12135, 12135], dtype=int64)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

numTypeCols = df.select_dtypes(include=[np.number]).columns.values
X = df[numTypeCols]
y = df["PLAYER_NAME"]
imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
imp = imp.fit(X)
XImp = imp.transform(X)

X2 = prospects[numTypeCols]

imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
imp = imp.fit(X2)
X2Imp = imp.transform(X2)


In [23]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(XImp,y)

predictions = dtree.predict(X2Imp)
print(prospects['PLAYER_NAME'].shape)
predictions.shape

(69,)


(69,)

In [18]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

ValueError: Found input variables with inconsistent numbers of samples: [20, 69]